In [ ]:
# Vérification des identifiants de connexion
def verify_login(identifier, password):
    """
    Vérifie si l'email/username et le mot de passe correspondent
    identifier peut être soit email soit username
    """
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    
    # Chercher l'utilisateur par email OU username
    c.execute(
        "SELECT id, username, password FROM users WHERE email = ? OR username = ?",
        (identifier, identifier)
    )
    user = c.fetchone()
    conn.close()
    
    if user and check_password_hash(user[2], password):
        return user[0], user[1]  # user_id, username
    return None, None

# Route de connexion
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Récupération des données du formulaire
        identifier = request.form.get('identifier', '').strip()
        password = request.form.get('password', '')
        
        # Vérification des champs obligatoires
        if not identifier:
            flash("L'email ou le nom d'utilisateur est obligatoire", 'error')
            return render_template('login.html')
        
        if not password:
            flash("Le mot de passe est obligatoire", 'error')
            return render_template('login.html')
        
        # Vérification des identifiants
        user_id, username = verify_login(identifier, password)
        
        if user_id:
            # Connexion réussie
            session['user_id'] = user_id
            session['username'] = username
            flash(f"Connexion réussie ! Bienvenue {username}", 'success')
            return redirect(url_for('timeline'))
        else:
            # Identifiants incorrects
            flash("Email/nom d'utilisateur ou mot de passe incorrect", 'error')
            return render_template('login.html')
    
    return render_template('login.html')

@app.route('/')
def home():
    # Redirige vers la page de connexion par défaut
    return redirect(url_for('login'))

@app.route('/timeline')
def timeline():
    if 'user_id' not in session:
        flash("Veuillez vous connecter pour accéder à votre timeline", 'error')
        return redirect(url_for('login'))
    
    # Récupérer plus d'informations utilisateur si nécessaire
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute("SELECT email, created_at FROM users WHERE id = ?", (session['user_id'],))
    user_info = c.fetchone()
    conn.close()
    
    return render_template('timeline.html', 
                         username=session.get('username'),
                         email=user_info[0] if user_info else '',
                         join_date=user_info[1] if user_info else '')



